In [1]:
%load_ext lab_black
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns
from glob import glob
import IPython.display as ipd
from itertools import cycle
from typing import TYPE_CHECKING
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split

import sys

sys.path.append("../src")
from Dataset import Dataset

if TYPE_CHECKING:
    plt.rcParams = {}
color_pal = plt.rcParams["axes.prop_cycle"].by_key()["color"]
color_cycle = cycle(plt.rcParams["axes.prop_cycle"].by_key()["color"])
sns.set_theme(style="white", palette=None)

/opt/conda/envs/kaggle/lib/python3.8/site-packages/xgboost/compat.py:31: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


## 训练集
* 每个 sequence 对应 60 行数据。 要根据这 60 行数据做分类。
* 这 60 行是某个 subject 的 60 个 step。
* 每个 step 有 13 个传感器值。

## 直接把60行拼一起，扔给 xgb 试试

In [195]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split

sensor_cols = [f"sensor{i:02d}" for i in range(13)]


def preprocess_df(df_):
    return (
        df_.pivot(
            index=["sequence", "subject"],
            columns=["step"],
            values=sensor_cols,
        )
        .reset_index()
        .set_index("sequence")
    )


def submit_result(pred):
    submit = pd.read_csv("dataset/csv/sample_submission.csv")
    print(submit.shape)
    submit.iloc[:, 1] = pred
    submit.to_csv("./submit.csv", index=False)


df_test = preprocess_df(test)

it's only .625 score

In [166]:
df = preprocess_df(train)
print(train.shape, train.sequence.nunique())
print(df.shape, df.index.nunique())

X_train, X_val, y_train, y_val = train_test_split(
    df, label.state, test_size=0.2, random_state=42
)
model = XGBClassifier(n_estimators=100, random_state=4, eval_metric="logloss")

model.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_val, y_val)], verbose=True)

/opt/conda/envs/kaggle/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/opt/conda/envs/kaggle/lib/python3.8/site-packages/xgboost/data.py:208: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


[0]	validation_0-logloss:0.66429	validation_1-logloss:0.67393
[1]	validation_0-logloss:0.64545	validation_1-logloss:0.66797
[2]	validation_0-logloss:0.63114	validation_1-logloss:0.65976
[3]	validation_0-logloss:0.61868	validation_1-logloss:0.65764
[4]	validation_0-logloss:0.60883	validation_1-logloss:0.65500
[5]	validation_0-logloss:0.59807	validation_1-logloss:0.65391
[6]	validation_0-logloss:0.58622	validation_1-logloss:0.64851
[7]	validation_0-logloss:0.57373	validation_1-logloss:0.64627
[8]	validation_0-logloss:0.56621	validation_1-logloss:0.64537
[9]	validation_0-logloss:0.55737	validation_1-logloss:0.64355
[10]	validation_0-logloss:0.54604	validation_1-logloss:0.64130
[11]	validation_0-logloss:0.53904	validation_1-logloss:0.64031
[12]	validation_0-logloss:0.53014	validation_1-logloss:0.63688
[13]	validation_0-logloss:0.52303	validation_1-logloss:0.63518
[14]	validation_0-logloss:0.51479	validation_1-logloss:0.63480
[15]	validation_0-logloss:0.50985	validation_1-logloss:0.63432
[1

In [190]:
pred = model.predict(df_test)
submit_result(pred)

(12218,)
(12218, 2)


# 有一些异常值

try to drop those Outliers and retry

got .633 score, little improve

In [209]:
train_ = train
for i in sensor_cols:
    p = 0.9999
    a = train_[i].quantile(1 - p)
    b = train_[i].quantile(p)
    train_ = train_.query(f"@a < {i} < @b")
print(f"{train_.shape[0] / train.shape[0] * 100:.2f}")

# fig, ax = plt.subplots(1, 1, figsize=(20, 20))
# train_.drop(columns=["sequence", "step"]).hist(ax=ax, bins=60)
# plt.tight_layout()


df = preprocess_df(train_)

X_train, X_val, y_train, y_val = train_test_split(
    df,
    label.query("sequence.isin(@df.index.values)").state,
    test_size=0.2,
    random_state=42,
)
model = XGBClassifier(n_estimators=70, random_state=42, eval_metric="logloss")

model.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_val, y_val)], verbose=True)

submit_result(model.predict(df_test))

99.09


/opt/conda/envs/kaggle/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/opt/conda/envs/kaggle/lib/python3.8/site-packages/xgboost/data.py:208: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


[0]	validation_0-logloss:0.66390	validation_1-logloss:0.67285
[1]	validation_0-logloss:0.64435	validation_1-logloss:0.66152
[2]	validation_0-logloss:0.62713	validation_1-logloss:0.65622
[3]	validation_0-logloss:0.61094	validation_1-logloss:0.64698
[4]	validation_0-logloss:0.59946	validation_1-logloss:0.64365
[5]	validation_0-logloss:0.58829	validation_1-logloss:0.64222
[6]	validation_0-logloss:0.58105	validation_1-logloss:0.63911
[7]	validation_0-logloss:0.57152	validation_1-logloss:0.63696
[8]	validation_0-logloss:0.56042	validation_1-logloss:0.63238
[9]	validation_0-logloss:0.55356	validation_1-logloss:0.63126
[10]	validation_0-logloss:0.54197	validation_1-logloss:0.62851
[11]	validation_0-logloss:0.53560	validation_1-logloss:0.62854
[12]	validation_0-logloss:0.52649	validation_1-logloss:0.62815
[13]	validation_0-logloss:0.52092	validation_1-logloss:0.62761
[14]	validation_0-logloss:0.51414	validation_1-logloss:0.62642
[15]	validation_0-logloss:0.50501	validation_1-logloss:0.62596
[1

how about lgbm,

it's .685 score! wow, I don't know why lgbm better than xgb so much.

In [251]:
from lightgbm import LGBMClassifier


train_ = train
for i in sensor_cols:
    p = 0.9999
    a = train_[i].quantile(1 - p)
    b = train_[i].quantile(p)
    train_ = train_.query(f"@a < {i} < @b")
print(f"{train_.shape[0] / train.shape[0] * 100:.2f}")

# fig, ax = plt.subplots(1, 1, figsize=(20, 20))
# train_.drop(columns=["sequence", "step"]).hist(ax=ax, bins=60)
# plt.tight_layout()


df = preprocess_df(train_)

X_train, X_val, y_train, y_val = train_test_split(
    df.values,
    label.query("sequence.isin(@df.index.values)").state,
    test_size=0.2,
    random_state=42,
)
model = LGBMClassifier(n_estimators=1000, random_state=42)

model.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_val, y_val)], verbose=True)

submit_result(model.predict(df_test))

99.09


/opt/conda/envs/kaggle/lib/python3.8/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[1]	training's binary_logloss: 0.683427	valid_1's binary_logloss: 0.684815
[2]	training's binary_logloss: 0.674809	valid_1's binary_logloss: 0.677939
[3]	training's binary_logloss: 0.6676	valid_1's binary_logloss: 0.671857
[4]	training's binary_logloss: 0.661037	valid_1's binary_logloss: 0.666603
[5]	training's binary_logloss: 0.65515	valid_1's binary_logloss: 0.662394
[6]	training's binary_logloss: 0.65038	valid_1's binary_logloss: 0.659596
[7]	training's binary_logloss: 0.644994	valid_1's binary_logloss: 0.655405
[8]	training's binary_logloss: 0.639937	valid_1's binary_logloss: 0.651306
[9]	training's binary_logloss: 0.636006	valid_1's binary_logloss: 0.649239
[10]	training's binary_logloss: 0.631591	valid_1's binary_logloss: 0.646927
[11]	training's binary_logloss: 0.625628	valid_1's binary_logloss: 0.641852
[12]	training's binary_logloss: 0.621927	valid_1's binary_logloss: 0.640018
[13]	training's binary_logloss: 0.617834	valid_1's binary_logloss: 0.637558
[14]	training's binary_lo

In [250]:
model.get_params()

{'boosting_type': 'gbdt',
 'class_weight': None,
 'colsample_bytree': 1.0,
 'importance_type': 'split',
 'learning_rate': 0.05,
 'max_depth': -1,
 'min_child_samples': 20,
 'min_child_weight': 0.001,
 'min_split_gain': 0.0,
 'n_estimators': 1000,
 'n_jobs': -1,
 'num_leaves': 31,
 'objective': None,
 'random_state': 42,
 'reg_alpha': 0.0,
 'reg_lambda': 0.0,
 'silent': 'warn',
 'subsample': 1.0,
 'subsample_for_bin': 200000,
 'subsample_freq': 0}